## 损失函数

损失函数是模型优化的目标，用于在众多的参数取值中，识别最理想的参数取值。
损失函数的计算在训练过程的代码中，每一轮模型训练的过程都相同，分如下三步：
1. 先根据输入数据正向计算预测输出；
2. 再根据预测值和真实值计算损失；
3. 最后根据损失反向传播梯度并更新参数；

In [1]:
"""交叉熵的代码实现

在读取数据部分，将标签的类型设置为int，体现它是一个标签而不是实数值（飞浆默认将标签处理成int64）
在网络定义部分，将输出层改成“输出10个标签的概率”的模式
在训练过程部分，将损失函数从均方误差(常用于回归问题)换成交叉熵（常用于分类问题）
"""

import numpy as np
import matplotlib.pyplot as plt

import paddle
from paddle.nn import Conv2D, MaxPool2D, Linear
import paddle.nn.functional as F

from PIL import Image
import os
import random
import gzip
import json

In [2]:
# 构建一个加载数据的迭代器
class MnistDataset(paddle.io.Dataset):
    def __init__(self, mode):
        # 读取原始数据
        datafile = '../datasets/mnist.json.gz'
        data = json.load(gzip.open(datafile))
        
        # 划分训练集、验证集和测试集
        train_set, valid_set, test_set = data
        
        # 数据集相关参数：图片宽、高度
        self.IMG_ROWS = 28
        self.IMG_COLS = 28
        
        # 拆分数据和标签
        if mode=='train':
            imgs, labels = train_set[0], train_set[1]
        elif mode=='valid':
            imgs, labels = valid_set[0], valid_set[1]
        elif mode=='test':
            imgs, labels = test_set[0], test_set[1]
        else:
            raise Exception("Mode can only be one of ['train', 'valid', 'test']")
                            
        # 校验数据
        imgs_length = len(imgs)
        assert len(imgs) == len(labels), \
            'Length of imgs({}) should be the same as labels({})'.format(len(imgs), len(labels))
                            
        self.imgs = imgs
        self.labels = labels
    
    def __getitem__(self, idx):
        img = np.reshape(self.imgs[idx], [1, self.IMG_ROWS, self.IMG_COLS]).astype('float32')
        label = np.reshape(self.labels[idx], [1]).astype('int64')
        
        return img, label
    
    def __len__(self):
        return len(self.imgs)

In [3]:
# 使用训练模式，迭代器每次迭代只返回batch=1的数据
train_dataset = MnistDataset(mode='train')

# 使用paddle.io.DataLoader，返回的是一个批次数据迭代器，并且是异步的
train_loader = paddle.io.DataLoader(train_dataset, batch_size=100, shuffle=True, drop_last=True)

# 加载验证数据集
valid_dataset = MnistDataset(mode='valid')
valid_loader = paddle.io.DataLoader(valid_dataset, batch_size=128, drop_last=True)

In [4]:
# 定义多层卷积神经网络
class MNIST(paddle.nn.Layer):
    def __init__(self):
        super(MNIST, self).__init__()

        # 定义卷积层，输出特征通道out_channels设置为20，卷积核的大小kernel_size为5，卷积步长stride=1，padding=2
        self.conv1 = Conv2D(in_channels=1, out_channels=20, kernel_size=5, stride=1, padding=2)
        # 定义池化层，池化核的大小kernel_size为2，池化步长为2
        self.max_pool1 = MaxPool2D(kernel_size=2, stride=2)
        # 定义卷积层，输出特征通道out_channels设置为20，卷积核的大小kernel_size为5，卷积步长stride=1，padding=2
        self.conv2 = Conv2D(in_channels=20, out_channels=20, kernel_size=5, stride=1, padding=2)
        # 定义池化层，池化核的大小kernel_size为2，池化步长为2
        self.max_pool2 = MaxPool2D(kernel_size=2, stride=2)
        # 定义一层全连接层，输出维度是10
        self.fc = Linear(in_features=980, out_features=10)
        
    # 定义网络前向计算过程，卷积后紧接着使用池化层，最后使用全连接层计算最终输出
    # 卷积层激活函数使用Relu，全连接层激活函数使用softmax
    def forward(self, inputs):
        x = self.conv1(inputs)
        x = F.relu(x)
        x = self.max_pool1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.max_pool2(x)
        x = paddle.reshape(x, [x.shape[0], 980])
        x = self.fc(x)
        return x

In [5]:
# 修改计算损失函数
def evaluation(model, datasets):
    model.eval()
    
    acc_set = list()
    for batch_id, data in enumerate(datasets()):
        images, labels = data
        images = paddle.to_tensor(images)
        labels = paddle.to_tensor(labels)
        pred = model(images)
        acc = paddle.metric.accuracy(input=pred, label=labels)
        acc_set.extend(acc.numpy())
    
    # 计算多个批次的准确率
    acc_val_mean = np.array(acc_set).mean()
    return acc_val_mean

In [6]:
# Change loss function from F.square_error_cost to F.cross_entropy
def train(model):
    model.train()
    
    # 调用加载数据的函数
    opt = paddle.optimizer.SGD(learning_rate=0.01, parameters=model.parameters())
    
    EPOCH_NUM = 10
    for epoch_id in range(EPOCH_NUM):
        for batch_id, data in enumerate(train_loader()):
            # 准备数据
            images, labels = data
            images = paddle.to_tensor(images)
            labels = paddle.to_tensor(labels)
            # 前向计算的过程
            predicts = model(images)
            
            # 计算损失，使用交叉熵损失函数，取一个批次样本损失的平均值
            loss = F.cross_entropy(predicts, labels)
            avg_loss = paddle.mean(loss)
            
            # 每训练了200批次的数据，打印下当前Loss的情况
            if batch_id % 200 == 0:
                print("epoch: {}, batch: {}, loss is: {}".format(epoch_id, batch_id, avg_loss.numpy()))
            
            # 后向传播，更新参数的过程
            avg_loss.backward()
            # 最小化loss,更新参数
            opt.step()
            # 清除梯度
            opt.clear_grad()
            
        acc_train_mean = evaluation(model, train_loader)
        acc_valid_mean = evaluation(model, valid_loader)
        print('train_acc: {}, valid acc: {}'.format(acc_train_mean, acc_valid_mean))
        
    #保存模型参数
    paddle.save(model.state_dict(), '../models/mnist.pdparams')

In [7]:
# Init Model
model = MNIST()

# Start training
train(model)

epoch: 0, batch: 0, loss is: [3.7077954]
epoch: 0, batch: 200, loss is: [0.20742299]
epoch: 0, batch: 400, loss is: [0.26519164]
train_acc: 0.9392999410629272, valid acc: 0.9464142918586731
epoch: 1, batch: 0, loss is: [0.21583277]
epoch: 1, batch: 200, loss is: [0.3002791]
epoch: 1, batch: 400, loss is: [0.06760629]
train_acc: 0.9623000025749207, valid acc: 0.9683493375778198
epoch: 2, batch: 0, loss is: [0.1559651]
epoch: 2, batch: 200, loss is: [0.10081352]
epoch: 2, batch: 400, loss is: [0.06157354]
train_acc: 0.9690800309181213, valid acc: 0.9719551205635071
epoch: 3, batch: 0, loss is: [0.1323459]
epoch: 3, batch: 200, loss is: [0.17427988]
epoch: 3, batch: 400, loss is: [0.04494239]
train_acc: 0.97107994556427, valid acc: 0.9741586446762085
epoch: 4, batch: 0, loss is: [0.12689942]
epoch: 4, batch: 200, loss is: [0.03878341]
epoch: 4, batch: 400, loss is: [0.14332902]
train_acc: 0.976099967956543, valid acc: 0.9777644276618958
epoch: 5, batch: 0, loss is: [0.05786611]
epoch: 5, 

In [13]:
# Load a local image, and convert format to model
def load_image(img_path):
    img = Image.open(img_path).convert('L')
    img = img.resize((28, 28))
    img = np.array(img).reshape(1, 1, 28, 28).astype(np.float32)
    # 图像归一化
    img = 1.0 - img / 255
    return img

In [32]:
model = MNIST()
model_params_path = '../models/mnist.pdparams'
img_path = '../datasets/mnist_example_6.jpg'

# Load model's params
param_dict = paddle.load(model_params_path)
model.load_dict(param_dict)

model.eval()
tensor_img = load_image(img_path)

# 模型返回10个分类标签的对应概率
results = model(paddle.to_tensor(tensor_img))

# 取出概率最大的标签作为预测输出
label = np.argsort(results.numpy())

print('识别结果：', label[0][-1])

识别结果： 6
